In [22]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import numpy as np

team_number = 20
url1 = "https://fantasy.premierleague.com/api/bootstrap-static/"
url2 = "https://fantasy.premierleague.com/api/entry/698498/history/"
url3 = "https://fantasy.premierleague.com/api/event/6/live/"
url4 = "https://fantasy.premierleague.com/api/fixtures"

#NaNs to zeros
def toint(a):
    if np.isnan(a):
        return 0
    else: return int(a)

#If no matches played not to devide by zero
def noZ(a):
    b = a.copy()
    for i in range(len(b)):
        if b[i] == 0:
            b[i]=1
    return b


p1 = requests.get(url1)
page1 = BeautifulSoup(p1.text)
data1 = str(page1.p)[3:-4]
d1 = json.loads(data1)
bigTable = pd.DataFrame(d1['elements'])
bigTable = bigTable[['team', 'element_type', 'web_name', 'goals_scored', 'assists', 'bonus', 'event_points', 'total_points', 
               'saves', 'own_goals', 'clean_sheets', 'penalties_missed', 'penalties_saved', 'yellow_cards', 'red_cards', 
               'minutes', 'bps', 'creativity', 'threat', 'ict_index', 'influence',
               'value_season', 'form', 'value_form', 'points_per_game', 
               'goals_conceded', 
               'in_dreamteam', 'dreamteam_count',
               'now_cost', 'cost_change_event', 'cost_change_event_fall',
               'cost_change_start', 'cost_change_start_fall', 'selected_by_percent',
               'transfers_in_event', 'transfers_out_event', 'transfers_in', 'transfers_out', 
               'chance_of_playing_this_round', 'chance_of_playing_next_round', 'news_added', 'news', 'status', 
               'ep_this', 'ep_next', 'first_name', 'second_name', 'team_code', 'id', 'photo', 'special', 'squad_number', 'code']]
bigTable['full_name'] = bigTable['first_name']+' '+bigTable['second_name']
bigTable.to_excel('fpltable.xlsx')

p4 = requests.get(url4)
page4 = BeautifulSoup(p4.text)
data4 = str(page4.p)[3:-4]
d4 = json.loads(data4)
Fixtures = pd.DataFrame(d4)
Fixtures.to_excel('fplfixtures.xlsx')

#Figuring out ended lastGameweek   lastGW

if Fixtures.at[1,'minutes'] != 90:
    lastGW = 0
elif Fixtures.at[len(Fixtures)-1,'minutes'] == 90:
    lastGW = Fixtures.at[len(Fixtures)-1,'event']
else:
    for i in range(1,len(Fixtures)):
        if Fixtures.at[i,'minutes'] != 90:
            lastGW = Fixtures.at[i-1,'event']
            break
lastGW = int(lastGW)

Gameweeks = pd.DataFrame()
for i in range(1,2*team_number - 1):
    url = "https://fantasy.premierleague.com/api/event/" + str(i) + "/live/"
    p = requests.get(url)
    page = BeautifulSoup(p.text)
    data = str(page.p)[3:-4]
    d = json.loads(data)
    nexTour = pd.DataFrame(d['elements'])
    
    if not nexTour.empty:
        nt1 = pd.DataFrame(nexTour['stats'].tolist())
        nt1['id'] = nexTour['id']
        nt1['gameweek'] = i
        nt1.index = nt1['gameweek']*1000+nt1['id']
        Gameweeks = Gameweeks.append(nt1)
        print(i)


teams = dict(zip(pd.DataFrame(d1['teams'])['id'],pd.DataFrame(d1['teams'])['name']))
players = dict(zip(bigTable['id'],bigTable['full_name']))
teamplayers = dict(zip(bigTable['id'],bigTable['team']))

Gameweeks['team'] = [teamplayers[i] for i in Gameweeks['id']]

Gameweeks['threat'] = pd.to_numeric(Gameweeks['threat'])
Gameweeks['creativity'] = pd.to_numeric(Gameweeks['creativity'])

Gameweeks['team_a'] = [int(Fixtures[(Fixtures['event'] == Gameweeks.iloc[i,20]) & \
                                         ((Fixtures['team_a'] == Gameweeks.iloc[i,21])|\
                                          (Fixtures['team_h'] == Gameweeks.iloc[i,21]))]['team_a']) \
                            for i in range(len(Gameweeks))]
Gameweeks['team_h'] = [int(Fixtures[(Fixtures['event'] == Gameweeks.iloc[i,20]) & \
                                         ((Fixtures['team_a'] == Gameweeks.iloc[i,21])|\
                                          (Fixtures['team_h'] == Gameweeks.iloc[i,21]))]['team_h']) \
                            for i in range(len(Gameweeks))]

Gameweeks['teamAgainst'] = [Gameweeks.at[i,'team_a'] if Gameweeks.at[i,'team'] == Gameweeks.at[i,'team_h']\
                            else Gameweeks.at[i,'team_h'] \
                           for i in Gameweeks.index]
Gameweeks['side'] = ['home' if Gameweeks.at[i,'team'] == Gameweeks.at[i,'team_h']\
                            else 'away' \
                           for i in Gameweeks.index]

del Gameweeks['team_a']
del Gameweeks['team_h']


Gameweeks.to_excel('fplgameweeks.xlsx')
Gameweeks

1
2
3
4
5
6
7
8
9
10
11
12
13
14


,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,in_dreamteam,influence,...,red_cards,saves,threat,total_points,yellow_cards,id,gameweek,team,teamAgainst,side
1014,0,1,27,1,0.0,0,0,1.8,False,18.0,...,0,2,0.0,7,0,14,1,1,13,away
1027,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0,0.0,0,0,27,1,2,17,away
1047,0,2,29,1,0.0,0,0,2.5,False,25.4,...,0,3,0.0,9,0,47,1,4,18,away
1048,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0,0.0,0,0,48,1,4,18,away
1072,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0,0.0,0,0,72,1,3,15,home
1073,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0,0.0,0,0,73,1,3,15,home
1074,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0,0.0,0,0,74,1,3,15,home
1093,0,1,28,1,10.0,0,0,3.6,False,25.6,...,0,3,0.0,8,0,93,1,5,16,home
1094,0,0,15,0,0.0,3,0,2.4,False,24.2,...,0,3,0.0,2,0,94,1,2,17,away
1095,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0,0.0,0,0,95,1,5,16,home


In [3]:
#NaNs to zeros
def toint(a):
    if np.isnan(a):
        return 0
    else: return int(a)

#If no matches played not to devide by zero
def noZ(a):
    b = a.copy()
    for i in range(len(b)):
        if b[i] == 0:
            b[i]=1
    return b
    
#Figuring out ended lastGameweek   lastGW

if Fixtures.at[1,'minutes'] != 90:
    lastGW = 0
elif Fixtures.at[len(Fixtures)-1,'minutes'] == 90:
    lastGW = Fixtures.at[len(Fixtures)-1,'event']
else:
    for i in range(1,len(Fixtures)):
        if Fixtures.at[i,'minutes'] != 90:
            lastGW = Fixtures.at[i-1,'event']
            break
lastGW = int(lastGW)



#Создание первой таблицы команд

TableTeams = pd.DataFrame(list(teams.keys()), columns = ['id'])
TableTeams['team'] = pd.DataFrame(list(teams.values()))

TableTeams['threat'] = [Gameweeks[Gameweeks['team']==i]['threat'].sum() for i in range(1,21)]
TableTeams['threat Adjusted'] = np.zeros(len(TableTeams))
TableTeams['Matches'] = np.zeros(len(TableTeams))
TableTeams['creativity'] = [Gameweeks[Gameweeks['team']==i]['creativity'].sum() for i in range(1,21)]
TableTeams['threatAllowed'] = [Gameweeks[Gameweeks['teamAgainst']==i]['threat'].sum() for i in range(1,21)]

#Threats and Threats Adjusted for GWs
threatAllowedAv = TableTeams['threatAllowed'].mean()



#Создание таблицы игроков

TablePlayers = pd.DataFrame(list(players.keys()), columns = ['id'])
TablePlayers['name'] = pd.DataFrame(list(players.values()))

TablePlayers['threat'] = [Gameweeks[Gameweeks['id']==TablePlayers.iloc[i,0]]['threat'].sum() for i in range(len(players))]
TablePlayers['threat Adjusted'] = np.zeros(len(TablePlayers))
TablePlayers['creativity'] = [Gameweeks[Gameweeks['id']==TablePlayers.iloc[i,0]]['creativity'].sum() \
                              for i in range(len(players))]

for j in range(lastGW,0,-1):
    TableTeams['threat GW'+str(j)] = [Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)]['threat'].sum() \
                                      for i in range(1,team_number+1)]
    TablePlayers['threat GW'+str(j)] = [Gameweeks[(Gameweeks['id']==TablePlayers.iat[i,0])&(Gameweeks['gameweek']==j)] \
                                    ['threat'].sum() \
                                    for i in range(len(TablePlayers))]
    TableTeams['threat GW'+str(j)+' Adj'] = [TableTeams.at[i-1,'threat GW'+str(j)]*threatAllowedAv/ \
                TableTeams.at[int(Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
                'threatAllowed'] \
                                             for i in range(1,team_number+1)]
    
    TablePlayers['threat GW'+str(j)+' Adj'] = [TablePlayers.at[i,'threat GW'+str(j)]*threatAllowedAv/ \
    TableTeams.at[toint(Gameweeks[(Gameweeks['id']==TablePlayers.iat[i,0])& \
                                (Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
                  'threatAllowed'] for i in range(len(TablePlayers))]
    
    TableTeams['threat Adjusted'] = TableTeams['threat Adjusted'] + TableTeams['threat GW'+str(j)+' Adj']
    TablePlayers['threat Adjusted'] = TablePlayers['threat Adjusted'] + TablePlayers['threat GW'+str(j)+' Adj']
    #pd.to_numeric(TableTeams['threat GW'+str(j)+' Adj'], downcast = 'integer')

    
#Запись таблиц

TableTeamsToWrite = TableTeams.copy()
del TableTeamsToWrite['id']
TableTeamsToWrite.sort_values('threat Adjusted', ascending = False, inplace = True)
TableTeamsToWrite = TableTeamsToWrite.reset_index(drop = True)
TableTeamsToWrite.to_excel('TableTeams.xlsx')

TablePlayersToWrite = TablePlayers.copy()
del TablePlayersToWrite['id']
TablePlayersToWrite.sort_values('threat Adjusted', ascending = False, inplace = True)
TablePlayersToWrite = TablePlayersToWrite.reset_index(drop = True)
TablePlayersToWrite.to_excel('TablePlayers.xlsx')



Gameweeks[Gameweeks['team']==5]['threat'].sum()
TableTeamsToWrite

,team,threat,threat Adjusted,Matches,creativity,threatAllowed,threat GW14,threat GW14 Adj,threat GW13,threat GW13 Adj,...,threat GW5,threat GW5 Adj,threat GW4,threat GW4 Adj,threat GW3,threat GW3 Adj,threat GW2,threat GW2 Adj,threat GW1,threat GW1 Adj
0,Man City,4445.0,4178.050085,0.0,3578.1,1582.0,332.0,268.297292,182.0,271.383220,...,382.0,267.013360,185.0,178.365975,317.0,277.114177,469.0,405.878947,237.0,212.798180
1,Liverpool,3193.0,3156.464204,0.0,2651.8,1607.0,235.0,226.572996,194.0,176.086143,...,318.0,256.983551,202.0,208.477455,362.0,304.556283,163.0,136.374974,222.0,155.175304
2,Chelsea,2987.0,3066.570646,0.0,2664.0,1621.0,235.0,211.002415,129.0,197.096018,...,229.0,241.604496,162.0,192.796750,287.0,200.609514,163.0,239.651642,160.0,225.633606
3,Everton,2727.0,2716.438000,0.0,2179.5,1952.0,143.0,210.246533,251.0,175.445951,...,209.0,182.703038,230.0,242.659537,131.0,93.376615,119.0,113.420702,126.0,114.365227
4,Leicester,2522.0,2530.685261,0.0,2363.3,1644.0,230.0,284.801742,243.0,234.286119,...,112.0,157.943524,181.0,158.226076,130.0,154.713442,113.0,168.496175,130.0,137.155391
5,Arsenal,2582.0,2438.747195,0.0,2240.9,2873.0,350.0,244.645749,219.0,183.227726,...,102.0,97.217744,304.0,263.085714,88.0,132.361419,226.0,233.247054,149.0,120.410532
6,Man Utd,2455.0,2351.308559,0.0,2269.7,1714.0,221.0,157.528487,147.0,174.945199,...,91.0,133.793248,276.0,230.917134,231.0,209.669583,110.0,116.054561,190.0,283.312153
7,Wolves,2268.0,2350.799078,0.0,1825.2,2291.0,215.0,255.872230,188.0,164.345316,...,170.0,253.489821,184.0,227.841393,180.0,185.771990,84.0,118.457643,90.0,132.322993
8,Southampton,2134.0,2331.656620,0.0,1669.0,2889.0,188.0,179.185647,267.0,224.631291,...,152.0,180.895716,126.0,177.686464,174.0,167.760431,178.0,267.731052,146.0,150.681725
9,Aston Villa,2319.0,2305.741824,0.0,2028.9,3391.0,111.0,156.533314,242.0,195.566098,...,107.0,96.073440,107.0,97.119677,143.0,177.072387,248.0,216.795949,86.0,74.425564


In [48]:
#Team tables

print(0)
#1. Creating  a table with average threat and GW threats for teams

TeamThreat = pd.DataFrame()
TeamThreat['id'] = pd.DataFrame(d1['teams'])['id']
TeamThreat['Team'] = pd.DataFrame(d1['teams'])['name']
TeamThreat['Threat av'] = np.zeros(len(TeamThreat))
TeamThreat['Matches'] = np.zeros(len(TeamThreat))

for j in range(lastGW,0,-1):
    TeamThreat['Threat GW'+str(j)] = [Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)]['threat'].sum() \
                                      for i in range(1,team_number+1)]
    TeamThreat['Matches'] = TeamThreat['Matches'] + (TeamThreat['Threat GW'+str(j)] != np.zeros(len(TeamThreat)))
    

TeamThreat['Threat av'] = [Gameweeks[Gameweeks['team']==i]['threat'].sum() for i in range(1,team_number+1)] \
    /noZ(TeamThreat['Matches'])
print(1)
#2. Creating  a table with average creativity and GW creativities for teams

TeamCreativity = pd.DataFrame()
TeamCreativity['id'] = pd.DataFrame(d1['teams'])['id']
TeamCreativity['Team'] = pd.DataFrame(d1['teams'])['name']
TeamCreativity['Creativity av'] = np.zeros(len(TeamCreativity))
TeamCreativity['Matches'] = np.zeros(len(TeamCreativity))

for j in range(lastGW,0,-1):
    TeamCreativity['Creativity GW'+str(j)] = [Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)] \
            ['creativity'].sum() for i in range(1,team_number+1)]
    TeamCreativity['Matches'] = TeamCreativity['Matches'] + \
            (TeamCreativity['Creativity GW'+str(j)] != np.zeros(len(TeamCreativity)))
    

TeamCreativity['Creativity av'] = [Gameweeks[Gameweeks['team']==i]['creativity'].sum() for i in range(1,team_number+1)] \
        /noZ(TeamCreativity['Matches'])
print(2)
#3. Creating  a table with average threat allowed by teams and GW threat allowed

TableDefence = pd.DataFrame()
TableDefence['id'] = pd.DataFrame(d1['teams'])['id']
TableDefence['Team'] = pd.DataFrame(d1['teams'])['name']
TableDefence['Threat allowed av'] = np.zeros(len(TableDefence))
TableDefence['Matches'] = np.zeros(len(TableDefence))

for j in range(lastGW,0,-1):
    TableDefence['Threat allowed GW'+str(j)] = [Gameweeks[(Gameweeks['teamAgainst']==i)&(Gameweeks['gameweek']==j)] \
            ['threat'].sum() for i in range(1,team_number+1)]
    TableDefence['Matches'] = TableDefence['Matches'] + \
            (TableDefence['Threat allowed GW'+str(j)] != np.zeros(len(TableDefence)))

TableDefence['Threat allowed av'] = [Gameweeks[Gameweeks['teamAgainst']==i]['threat'].sum() for i in range(1,team_number+1)] \
        /noZ(TableDefence['Matches'])

threatAllowedAv = TableDefence['Threat allowed av'].mean()
print(3)
#4. Creating  a table with average adjusted threat and GW threats adj for teams

TeamThreatAd = pd.DataFrame()
TeamThreatAd['id'] = pd.DataFrame(d1['teams'])['id']
TeamThreatAd['Team'] = pd.DataFrame(d1['teams'])['name']
TeamThreatAd['Threat av adj'] = np.zeros(len(TeamThreatAd))
TeamThreatAd['Matches'] = np.zeros(len(TeamThreatAd))

for j in range(lastGW,0,-1):    
    TeamThreatAd['Threat GW'+str(j)+' adj'] = [TeamThreat.at[i-1,'Threat GW'+str(j)]*threatAllowedAv/ \
                TableDefence.at[int(Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
                'Threat allowed av'] \
                                             for i in range(1,team_number+1)]
    TeamThreatAd['Matches'] = TeamThreatAd['Matches'] + (TeamThreatAd['Threat GW'+str(j)+' adj'] != np.zeros(len(TeamThreat)))
    TeamThreatAd['Threat av adj'] = TeamThreatAd['Threat av adj']  + TeamThreatAd['Threat GW'+str(j)+' adj']

TeamThreatAd['Threat av adj'] = TeamThreatAd['Threat av adj']/noZ(TeamThreatAd['Matches'])
print(4)
#5. Creating  a table with average adjusted creativity and GW creativities adj for teams

TeamCreativityAd = pd.DataFrame()
TeamCreativityAd['id'] = pd.DataFrame(d1['teams'])['id']
TeamCreativityAd['Team'] = pd.DataFrame(d1['teams'])['name']
TeamCreativityAd['Creativity av adj'] = np.zeros(len(TeamCreativityAd))
TeamCreativityAd['Matches'] = np.zeros(len(TeamCreativityAd))

for j in range(lastGW,0,-1):    
    TeamCreativityAd['Creativity GW'+str(j)+' adj'] = [TeamCreativity.at[i-1,'Creativity GW'+str(j)]*threatAllowedAv/ \
                TableDefence.at[int(Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
                'Threat allowed av'] \
                                             for i in range(1,team_number+1)]
    TeamCreativityAd['Matches'] = TeamCreativityAd['Matches'] + (TeamCreativityAd['Creativity GW'+str(j)+' adj'] != \
                                                                 np.zeros(len(TeamCreativity)))
    TeamCreativityAd['Creativity av adj'] = TeamCreativityAd['Creativity av adj']  + \
        TeamCreativityAd['Creativity GW'+str(j)+' adj']

TeamCreativityAd['Creativity av adj'] = TeamCreativityAd['Creativity av adj']/noZ(TeamThreatAd['Matches'])
print(5)
#6. Creating  a table with average threat allowed adjusted by teams and GW threat allowed adjusted

TableDefenceAd = pd.DataFrame()
TableDefenceAd['id'] = pd.DataFrame(d1['teams'])['id']
TableDefenceAd['Team'] = pd.DataFrame(d1['teams'])['name']
TableDefenceAd['Threat allowed av adj'] = np.zeros(len(TableDefenceAd))
TableDefenceAd['Matches'] = np.zeros(len(TableDefenceAd))

for j in range(lastGW,0,-1):    
    TableDefenceAd['Threat allowed GW'+str(j)+' adj'] = [TableDefence.at[i-1,'Threat allowed GW'+str(j)]*threatAllowedAv/ \
                TeamThreat.at[int(Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
                'Threat av'] \
                                             for i in range(1,team_number+1)]
    TableDefenceAd['Matches'] = TableDefenceAd['Matches'] + (TableDefenceAd['Threat allowed GW'+str(j)+' adj'] != \
                                                                 np.zeros(len(TableDefenceAd)))
    TableDefenceAd['Threat allowed av adj'] = TableDefenceAd['Threat allowed av adj']  + \
        TableDefenceAd['Threat allowed GW'+str(j)+' adj']

TableDefenceAd['Threat allowed av adj'] = TableDefenceAd['Threat allowed av adj']/noZ(TableDefenceAd['Matches'])

print(6)


#Total Team Table

TableTeams = pd.DataFrame()
TableTeams['id'] = pd.DataFrame(d1['teams'])['id']
TableTeams['Team'] = pd.DataFrame(d1['teams'])['name']

TableTeams['Threat adjusted'] = TeamThreatAd['Threat av adj']
TableTeams['Threat'] = TeamThreat['Threat av']
TableTeams['Creativity adjusted'] = TeamCreativityAd['Creativity av adj']
TableTeams['Creativity'] = TeamCreativity['Creativity av']
TableTeams['Threat allowed adjusted'] = TableDefenceAd['Threat allowed av adj']
TableTeams['Threat allowed'] = TableDefence['Threat allowed av']
print(7)



#PLayer Tables

#PlayerMatches = pd.DataFrame(list(players.keys()), columns = ['id'])
PlayerMatches = pd.DataFrame()
PlayerMatches['id'] = bigTable['id']
PlayerMatches['Team number'] = [bigTable[bigTable['id'] == i]['team'].sum() for i in players.keys()]
PlayerMatches['Team'] = [teams[PlayerMatches.at[i,'Team number']] for i in range(len(players))]
PlayerMatches['Team games'] = np.zeros(len(players))
PlayerMatches['Played'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
    PlayerMatches['Played'] = PlayerMatches['Played'] + ([Gameweeks[(Gameweeks['id']==PlayerMatches.iat[i,0])&\
                                    (Gameweeks['gameweek']==j)]['minutes'].sum() \
                                    for i in range(len(PlayerMatches))] != np.zeros(len(PlayerMatches)))

PlayerMatches['Team games'] = PlayerMatches['Team games'] + [len(Gameweeks[Gameweeks['id']==PlayerMatches.iat[i,0]])\
                                    for i in range(len(PlayerMatches))]
print(8)
#1 Players Threat

PlayerThreat = pd.DataFrame()
PlayerThreat['id'] = bigTable['id']
PlayerThreat['Name'] = bigTable['full_name']
PlayerThreat['Team'] = PlayerMatches['Team']
PlayerThreat['Threat per fixture'] = np.zeros(len(players))
PlayerThreat['Threat per game'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
    PlayerThreat['Threat GW'+str(j)] = [Gameweeks[(Gameweeks['id']==PlayerThreat.iat[i,0])&(Gameweeks['gameweek']==j)] \
                                    ['threat'].sum() \
                                    for i in range(len(players))]
    PlayerThreat['Threat per game'] = PlayerThreat['Threat per game'] + PlayerThreat['Threat GW'+str(j)]

PlayerThreat['Threat per fixture'] = PlayerThreat['Threat per game']/noZ(PlayerMatches['Team games'])
PlayerThreat['Threat per game'] = PlayerThreat['Threat per game']/noZ(PlayerMatches['Played'])
print(9)
#2 Players Creativity

PlayerCreativity = pd.DataFrame()
PlayerCreativity['id'] = bigTable['id']
PlayerCreativity['Name'] = bigTable['full_name']
PlayerCreativity['Team'] = PlayerMatches['Team']
PlayerCreativity['Creativity per fixture'] = np.zeros(len(players))
PlayerCreativity['Creativity per game'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
    PlayerCreativity['Creativity GW'+str(j)] = [Gameweeks[(Gameweeks['id']==PlayerThreat.iat[i,0])&\
                    (Gameweeks['gameweek']==j)]['creativity'].sum() for i in range(len(players))]
    PlayerCreativity['Creativity per game'] = PlayerCreativity['Creativity per game']\
        + PlayerCreativity['Creativity GW'+str(j)]

PlayerCreativity['Creativity per fixture'] = PlayerCreativity['Creativity per game']/noZ(PlayerMatches['Team games'])
PlayerCreativity['Creativity per game'] = PlayerCreativity['Creativity per game']/noZ(PlayerMatches['Played'])
print(10)
#3 Players Threat Adjusted

PlayerThreatAd = pd.DataFrame()
PlayerThreatAd['id'] = bigTable['id']
PlayerThreatAd['Name'] = bigTable['full_name']
PlayerThreatAd['Team number'] = PlayerMatches['Team number']
PlayerThreatAd['Team'] = PlayerMatches['Team']
PlayerThreatAd['Threat per fixture adj'] = np.zeros(len(players))
PlayerThreatAd['Threat per game adj'] = np.zeros(len(players))
for j in range(lastGW,0,-1):
        
    PlayerThreatAd['Threat GW'+str(j) + 'adj'] = [PlayerThreat.at[i,'Threat GW'+str(j)]*threatAllowedAv/ \
            TableDefence.at[toint(Gameweeks[(Gameweeks['team']==PlayerThreatAd.at[i,'Team number'])&\
            (Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
            'Threat allowed av']  for i in range(len(players))]                     
    
    PlayerThreatAd['Threat per game adj'] = PlayerThreatAd['Threat per game adj'] +\
            PlayerThreatAd['Threat GW'+str(j) + 'adj']
PlayerThreatAd['Threat per fixture adj'] = PlayerThreatAd['Threat per game adj']/noZ(PlayerMatches['Team games'])
PlayerThreatAd['Threat per game adj'] = PlayerThreatAd['Threat per game adj']/noZ(PlayerMatches['Played'])
print(11)

#4 PLayers Creativity Adjusted

PlayerCreativityAd = pd.DataFrame()
PlayerCreativityAd['id'] = bigTable['id']
PlayerCreativityAd['Name'] = bigTable['full_name']
PlayerCreativityAd['Team number'] = PlayerMatches['Team number']
PlayerCreativityAd['Team'] = PlayerMatches['Team']
PlayerCreativityAd['Creativity per fixture adj'] = np.zeros(len(players))
PlayerCreativityAd['Creativity per game adj'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
        
    PlayerCreativityAd['Creativity GW'+str(j) + 'adj'] = [PlayerCreativity.at[i,'Creativity GW'+str(j)]*threatAllowedAv/ \
            TableDefence.at[toint(Gameweeks[(Gameweeks['team']==PlayerCreativityAd.at[i,'Team number'])&\
                                          (Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
            'Threat allowed av']  for i in range(len(players))]                     
    
    PlayerCreativityAd['Creativity per game adj'] = PlayerCreativityAd['Creativity per game adj'] +\
            PlayerCreativityAd['Creativity GW'+str(j) + 'adj']

PlayerCreativityAd['Creativity per fixture adj'] = PlayerCreativityAd['Creativity per game adj']/\
            noZ(PlayerMatches['Team games'])
PlayerCreativityAd['Creativity per game adj'] = PlayerCreativityAd['Creativity per game adj']/\
            noZ(PlayerMatches['Played'])
print(12)


PlayerThreatAd

0
1
2
3
4
5
6
7
8
9
10
11
12


,id,Name,Team number,Team,Threat per fixture adj,Threat per game adj,Threat GW14adj,Threat GW13adj,Threat GW12adj,Threat GW11adj,Threat GW10adj,Threat GW9adj,Threat GW8adj,Threat GW7adj,Threat GW6adj,Threat GW5adj,Threat GW4adj,Threat GW3adj,Threat GW2adj,Threat GW1adj
0,1,Shkodran Mustafi,1,Arsenal,2.696096,37.745344,37.745344,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,Héctor Bellerín,1,Arsenal,0.764136,5.348951,0.000000,3.346625,7.351277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,3,Sead Kolasinac,1,Arsenal,3.622744,4.610765,2.795951,0.000000,17.643066,0.000000,0.000000,11.901034,0.000000,0.000000,7.127986,0.000000,11.250376,0.000000,0.000000,0.000000
3,4,Ainsley Maitland-Niles,1,Arsenal,1.937448,4.520713,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.702389,3.812461,10.384962,0.000000,7.224466,0.000000
4,5,Sokratis Papastathopoulos,1,Arsenal,4.409434,5.144339,0.000000,30.119626,0.000000,4.220166,17.245550,2.380207,0.000000,0.000000,5.702389,0.000000,0.000000,0.000000,2.064133,0.000000
5,6,Nacho Monreal,1,Arsenal,0.955514,4.459064,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.016428,4.128266,3.232497
6,7,Laurent Koscielny,1,Arsenal,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,8,Konstantinos Mavropanos,1,Arsenal,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,9,Carl Jenkinson,1,Arsenal,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,10,Rob Holding,1,Arsenal,0.210036,2.940511,0.000000,0.000000,2.940511,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [49]:
#Tables2Files

del TeamThreat['id']
TeamThreat.sort_values('Threat av', ascending = False, inplace = True)
TeamThreat.index = np.arange(1, len(TeamThreat) + 1)
TeamThreat.to_csv('TeamThreat.csv')

del TeamCreativity['id']
TeamCreativity.sort_values('Creativity av', ascending = False, inplace = True)
TeamCreativity.index = np.arange(1, len(TeamCreativity) + 1)
TeamCreativity.to_csv('TeamCreativity.csv')

del TableDefence['id']
TableDefence.sort_values('Threat allowed av', ascending = True, inplace = True)
TableDefence.index = np.arange(1, len(TableDefence) + 1)
TableDefence.to_csv('TableDefence.csv')

del TeamThreatAd['id']
TeamThreatAd.sort_values('Threat av adj', ascending = False, inplace = True)
TeamThreatAd.index = np.arange(1, len(TeamThreatAd) + 1)
TeamThreatAd.to_csv('TeamThreatAd.csv')

del TeamCreativityAd['id']
TeamCreativityAd.sort_values('Creativity av adj', ascending = False, inplace = True)
TeamCreativityAd.index = np.arange(1, len(TeamCreativityAd) + 1)
TeamCreativityAd.to_csv('TeamCreativityAd.csv')

del TableDefenceAd['id']
TableDefenceAd.sort_values('Threat allowed av adj', ascending = True, inplace = True)
TableDefenceAd.index = np.arange(1, len(TableDefenceAd) + 1)
TableDefenceAd.to_csv('TableDefenceAd.csv')

del TableTeams['id']
TableTeams.sort_values('Threat adjusted', ascending = False, inplace = True)
TableTeams.index = np.arange(1, len(TableTeams) + 1)
TableTeams.to_csv('TableTeams.csv')

del PlayerThreat['id']
PlayerThreat.sort_values('Threat per fixture', ascending = False, inplace = True)
PlayerThreat.index = np.arange(1, len(players) + 1)
PlayerThreat.to_csv('PlayerThreat.csv')

del PlayerCreativity['id']
PlayerCreativity.sort_values('Creativity per fixture', ascending = False, inplace = True)
PlayerCreativity.index = np.arange(1, len(players) + 1)
PlayerCreativity.to_csv('PlayerCreativity.csv')

del PlayerThreatAd['id']
del PlayerThreatAd['Team number']
PlayerThreatAd.sort_values('Threat per fixture adj', ascending = False, inplace = True)
PlayerThreatAd.index = np.arange(1, len(players) + 1)
PlayerThreatAd.to_csv('PlayerThreatAd.csv')

del PlayerCreativityAd['id']
del PlayerCreativityAd['Team number']
PlayerCreativityAd.sort_values('Creativity per fixture adj', ascending = False, inplace = True)
PlayerCreativityAd.index = np.arange(1, len(players) + 1)
PlayerCreativityAd.to_csv('PlayerCreativityAd.csv')

PlayerThreatAd

,Name,Team,Threat per fixture adj,Threat per game adj,Threat GW14adj,Threat GW13adj,Threat GW12adj,Threat GW11adj,Threat GW10adj,Threat GW9adj,Threat GW8adj,Threat GW7adj,Threat GW6adj,Threat GW5adj,Threat GW4adj,Threat GW3adj,Threat GW2adj,Threat GW1adj
1,Raheem Sterling,Man City,54.169794,58.336702,75.155567,8.946700,93.254636,38.486189,59.162283,32.675779,51.697032,94.108402,0.000000,66.403846,9.641404,56.821519,105.580451,66.443314
2,Roberto Firmino,Liverpool,52.830113,52.830113,34.709055,55.367293,44.308407,32.788735,97.791729,62.049242,60.280474,11.901034,86.484762,23.435607,74.308796,83.290254,26.773001,46.133198
3,Mohamed Salah,Liverpool,50.517826,58.937464,28.924212,0.000000,73.338053,24.947950,80.483459,0.000000,138.204015,80.927031,53.680197,58.993079,59.859863,56.368152,24.263032,27.260526
4,Raúl Jiménez,Wolves,45.978280,45.978280,122.580650,76.053418,29.224742,97.592621,10.505617,19.243094,45.836283,12.390497,24.506834,19.384516,66.866496,44.378864,31.024621,44.107664
5,Teemu Pukki,Norwich,42.319596,42.319596,23.556840,29.718443,47.655757,38.565616,22.563361,22.728608,49.895901,29.952798,45.410931,50.419912,21.549183,56.662431,92.126179,61.668388
6,Sergio Agüero,Man City,39.913363,46.565590,0.000000,52.189081,66.180709,92.868847,0.000000,0.000000,48.531907,8.667879,53.374448,62.209919,55.920144,59.444051,44.136090,15.264004
7,Marcus Rashford,Man Utd,39.768375,39.768375,14.968770,49.984343,88.700917,20.980253,56.618016,36.098569,1.616249,31.969997,5.387296,29.405109,98.725441,26.322155,9.495373,86.484762
8,Jamie Vardy,Leicester,39.653818,39.653818,85.440523,91.593339,40.383153,36.306421,77.809034,21.673399,15.041070,51.719960,38.943609,5.640840,39.337975,32.132792,14.911166,4.220166
9,Tammy Abraham,Chelsea,37.992264,40.914746,0.000000,32.085398,22.691513,72.436751,5.160333,50.911836,29.282970,74.238811,37.602676,60.137364,72.596307,33.551417,7.351277,33.845041
10,Danny Ings,Southampton,37.366696,37.366696,53.374448,21.874208,43.339395,10.695133,1.470255,86.513400,90.958112,38.943609,38.463797,2.380207,33.845041,12.533825,88.742315,0.000000


In [30]:
A = pd.DataFrame()
A['1'] = pd.DataFrame(d1['teams'])['name']
A

,1
0,Arsenal
1,Aston Villa
2,Bournemouth
3,Brighton
4,Burnley
5,Chelsea
6,Crystal Palace
7,Everton
8,Leicester
9,Liverpool


In [85]:
#PLayer Tables

PlayerMatches = pd.DataFrame(list(players.keys()), columns = ['id'])
PlayerMatches['Team'] = [teams[bigTable[bigTable['id'] == i]['team'].sum()] for i in players.keys()]
PlayerMatches['Team games'] = np.zeros(len(players))
PlayerMatches['Played'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
    PlayerMatches['Played'] = PlayerMatches['Played'] + ([Gameweeks[(Gameweeks['id']==PlayerThreat.iat[i,0])&\
                                    (Gameweeks['gameweek']==j)]['minutes'].sum() \
                                    for i in range(len(PlayerMatches))] != np.zeros(len(PlayerMatches)))
PlayerMatches['Team games'] = PlayerMatches['Team games'] + [len(Gameweeks[Gameweeks['id']==PlayerThreat.iat[i,0]])\
                                                                            for i in range(len(PlayerMatches))]

#1 Players Threat

PlayerThreat = pd.DataFrame(list(players.keys()), columns = ['id'])
PlayerThreat['Name'] = pd.DataFrame(list(players.values()))
PlayerThreat['Team'] = PlayerMatches['Team']
PlayerThreat['Threat per fixture'] = np.zeros(len(players))
PlayerThreat['Threat per game'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
    PlayerThreat['Threat GW'+str(j)] = [Gameweeks[(Gameweeks['id']==PlayerThreat.iat[i,0])&(Gameweeks['gameweek']==j)] \
                                    ['threat'].sum() \
                                    for i in range(len(players))]
    PlayerThreat['Threat per game'] = PlayerThreat['Threat per game'] + PlayerThreat['Threat GW'+str(j)]

PlayerThreat['Threat per fixture'] = PlayerThreat['Threat per game']/noZ(PlayerMatches['Team games'])
PlayerThreat['Threat per game'] = PlayerThreat['Threat per game']/noZ(PlayerMatches['Played'])

#2 Players Creativity

PlayerCreativity = pd.DataFrame(list(players.keys()), columns = ['id'])
PlayerCreativity['Name'] = pd.DataFrame(list(players.values()))
PlayerCreativity['Team'] = PlayerMatches['Team']
PlayerCreativity['Creativity per fixture'] = np.zeros(len(players))
PlayerCreativity['Creativity per game'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
    PlayerCreativity['Creativity GW'+str(j)] = [Gameweeks[(Gameweeks['id']==PlayerThreat.iat[i,0])&(Gameweeks['gameweek']==j)] \
                                    ['creativity'].sum() \
                                    for i in range(len(players))]
    PlayerCreativity['Creativity per game'] = PlayerCreativity['Creativity per game']\
        + PlayerCreativity['Creativity GW'+str(j)]

PlayerCreativity['Creativity per fixture'] = PlayerCreativity['Creativity per game']/noZ(PlayerMatches['Team games'])
PlayerCreativity['Creativity per game'] = PlayerCreativity['Creativity per game']/noZ(PlayerMatches['Played'])

#3 Players Threat Adjusted

PlayerThreatAd = pd.DataFrame(list(players.keys()), columns = ['id'])
PlayerThreatAd['Name'] = pd.DataFrame(list(players.values()))
PlayerThreatAd['Team'] = PlayerMatches['Team']
PlayerThreatAd['Threat per fixture adj'] = np.zeros(len(players))
PlayerThreatAd['Threat per game adj'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
        
    PlayerThreatAd['Threat GW'+str(j) + 'adj'] = [PlayerThreat.at[i,'Threat GW'+str(j)]*threatAllowedAv/ \
            TableDefence.at[toint(Gameweeks[(Gameweeks['team']==PlayerThreatAd.at[i,'Team'])&\
                                          (Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
            'Threat allowed av']  for i in range(len(players))]                     
    
    PlayerThreatAd['Threat per game adj'] = PlayerThreatAd['Threat per game adj'] +\
            PlayerThreatAd['Threat GW'+str(j) + 'adj']

PlayerThreatAd['Threat per fixture adj'] = PlayerThreatAd['Threat per game adj']/noZ(PlayerMatches['Team games'])
PlayerThreatAd['Threat per game adj'] = PlayerThreatAd['Threat per game adj']/noZ(PlayerMatches['Played'])


#4 Creativity Threat Adjusted

PlayerCreativityAd = pd.DataFrame(list(players.keys()), columns = ['id'])
PlayerCreativityAd['Name'] = pd.DataFrame(list(players.values()))
PlayerCreativityAd['Team'] = PlayerMatches['Team']
PlayerCreativityAd['Creativity per fixture adj'] = np.zeros(len(players))
PlayerCreativityAd['Creativity per game adj'] = np.zeros(len(players))

for j in range(lastGW,0,-1):
        
    PlayerCreativityAd['Creativity GW'+str(j) + 'adj'] = [PlayerCreativity.at[i,'Creativity GW'+str(j)]*threatAllowedAv/ \
            TableDefence.at[toint(Gameweeks[(Gameweeks['team']==PlayerCreativityAd.at[i,'Team'])&\
                                          (Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), \
            'Threat allowed av']  for i in range(len(players))]                     
    
    PlayerCreativityAd['Creativity per game adj'] = PlayerCreativityAd['Creativity per game adj'] +\
            PlayerCreativityAd['Creativity GW'+str(j) + 'adj']

PlayerCreativityAd['Creativity per fixture adj'] = PlayerCreativityAd['Creativity per game adj']/\
            noZ(PlayerMatches['Team games'])
PlayerCreativityAd['Creativity per game adj'] = PlayerCreativityAd['Creativity per game adj']/\
            noZ(PlayerMatches['Played'])



PlayerCreativityAd


,id,Name,Team,Creativity per fixture adj,Creativity per game adj,Creativity GW13adj,Creativity GW12adj,Creativity GW11adj,Creativity GW10adj,Creativity GW9adj,Creativity GW8adj,Creativity GW7adj,Creativity GW6adj,Creativity GW5adj,Creativity GW4adj,Creativity GW3adj,Creativity GW2adj,Creativity GW1adj
0,1,Shkodran Mustafi,Arsenal,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,Héctor Bellerín,Arsenal,1.326913,8.624936,4.833297,12.416574,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,3,Sead Kolasinac,Arsenal,7.147383,9.291597,0.000000,1.499989,0.249998,0.583329,27.166464,1.666654,1.916652,11.499914,19.666520,28.416455,0.000000,0.249998,0.000000
3,4,Ainsley Maitland-Niles,Arsenal,6.935846,15.027666,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.499922,10.166591,34.749741,7.999940,16.499877,10.249924
4,5,Sokratis Papastathopoulos,Arsenal,1.916652,2.076373,3.666639,0.000000,0.333331,0.249998,0.666662,0.583329,0.249998,8.749935,0.166665,0.166665,0.249998,9.499929,0.333331
5,6,Nacho Monreal,Arsenal,2.615365,11.333249,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.999993,19.999851,12.999903
6,7,Laurent Koscielny,Arsenal,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,8,Konstantinos Mavropanos,Arsenal,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,9,Carl Jenkinson,Arsenal,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,10,Rob Holding,Arsenal,0.025641,0.333331,0.000000,0.333331,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [80]:
#Figuring out ended lastGameweek   lastGW
if Fixtures.at[1,'started'] == False:
    lastGW = 0
elif Fixtures.at[len(Fixtures)-1,'started'] == True:
    lastGW = Fixtures.at[len(Fixtures)-1,'event']
else:
    for i in range(1,len(Fixtures)):
        if Fixtures.at[i,'started']==False:
            lastGW = Fixtures.at[i-1,'event']
            break
lastGW = int(lastGW)

#Threats and THreats Adjusted for GWs
threatAllowedAv = TableTeams['threatAllowed'].mean()
TableTeams['threat Adjusted'] = np.zeros(len(TableTeams))

for j in range(lastGW,0,-1):
    TableTeams['threat GW'+str(j)] = [Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)]['threat'].sum() \
                                      for i in range(1,team_number+1)]
    TableTeams['threat GW'+str(j)+' Adj'] = [TableTeams.at[i-1,'threat GW'+str(j)]*threatAllowedAv/ \
    TableTeams.at[int(Gameweeks[(Gameweeks['team']==i)&(Gameweeks['gameweek']==j)]['teamAgainst'].mean()-1), 'threatAllowed'] \
                                             for i in range(1,team_number+1)]
    TableTeams['threat Adjusted'] = TableTeams['threat Adjusted'] + TableTeams['threat GW'+str(j)+' Adj']
    #pd.to_numeric(TableTeams['threat GW'+str(j)+' Adj'], downcast = 'integer')
TableTeams

,team,threat,threat Adjusted,creativity,threatAllowed,threat GW11,threat GW11 Adj,threat GW10,threat GW10 Adj,threat GW9,...,threat GW5,threat GW5 Adj,threat GW4,threat GW4 Adj,threat GW3,threat GW3 Adj,threat GW2,threat GW2 Adj,threat GW1,threat GW1 Adj
0,Arsenal,1916.0,1848.058742,1805.9,2199.0,137.0,133.236225,189.0,176.145186,127.0,...,102.0,96.791764,304.0,247.462852,88.0,151.059963,226.0,216.871078,149.0,126.042590
1,Aston Villa,1841.0,1876.777018,1593.1,2806.0,84.0,144.193601,160.0,239.419920,318.0,...,107.0,97.828944,107.0,99.722407,143.0,183.810301,248.0,221.776952,86.0,70.005938
2,Bournemouth,1661.0,1803.577471,1289.3,2100.0,188.0,259.217768,165.0,156.574912,146.0,...,169.0,217.230356,98.0,141.459723,173.0,258.872789,154.0,103.066393,182.0,226.348940
3,Brighton,1853.0,1680.124188,1720.4,1718.0,270.0,185.935644,94.0,120.826352,238.0,...,181.0,173.688784,111.0,166.097570,180.0,149.637450,167.0,152.686295,71.0,67.374659
4,Burnley,1530.0,1573.584099,1355.4,1957.0,44.0,54.721722,158.0,227.543021,164.0,...,36.0,39.351688,60.0,102.995430,157.0,152.686769,227.0,193.858413,160.0,133.011067
5,Chelsea,2364.0,2456.532307,2114.1,1304.0,260.0,246.724103,245.0,235.103602,259.0,...,229.0,222.708726,162.0,201.475430,287.0,197.642703,163.0,235.285050,160.0,220.610866
6,Crystal Palace,1769.0,1740.567849,1191.9,2015.0,112.0,161.668255,190.0,162.260346,84.0,...,174.0,141.639922,251.0,167.984836,111.0,153.048789,88.0,109.443444,179.0,230.084223
7,Everton,1992.0,2039.562641,1590.1,1461.0,118.0,96.054660,143.0,156.313650,300.0,...,209.0,186.900738,230.0,223.681253,131.0,87.673361,119.0,112.923724,126.0,117.430124
8,Leicester,1831.0,1815.089585,1763.3,1301.0,171.0,159.369454,392.0,325.877114,173.0,...,112.0,154.427606,181.0,161.861405,130.0,161.677815,113.0,162.736465,130.0,126.428534
9,Liverpool,2606.0,2469.720126,2252.8,1094.0,348.0,232.903279,343.0,279.209731,117.0,...,318.0,269.003649,202.0,193.840521,362.0,309.148658,163.0,135.505024,222.0,152.880418


In [78]:
TableTeams['threat Adjusted'].sum()

37559.0

In [45]:
a = Gameweeks.groupby('team')
a.size()
arsenal = a.get_group(1)
pd.set_option('precision',0)
a.agg({'threat': 'sum', 'creativity': 'sum'}).sort_values(by = 'threat', ascending=False)

,threat,creativity
team,,
11,3232,2697
10,2258,1955
6,2104,1914
8,1874,1471
1,1779,1645
2,1757,1522
19,1727,1414
9,1660,1588
7,1657,1116


In [21]:
Gameweeks['team_a'] = [int(Fixtures[(Fixtures['event'] == Gameweeks.iloc[i,20]) & \
                                         ((Fixtures['team_a'] == Gameweeks.iloc[i,21])|\
                                          (Fixtures['team_h'] == Gameweeks.iloc[i,21]))]['team_a']) \
                            for i in range(len(Gameweeks))]
Gameweeks['team_h'] = [int(Fixtures[(Fixtures['event'] == Gameweeks.iloc[i,20]) & \
                                         ((Fixtures['team_a'] == Gameweeks.iloc[i,21])|\
                                          (Fixtures['team_h'] == Gameweeks.iloc[i,21]))]['team_h']) \
                            for i in range(len(Gameweeks))]

Gameweeks['teamAgainst'] = [Gameweeks.at[i,'team_a'] if Gameweeks.at[i,'team'] == Gameweeks.at[i,'team_h']\
                            else Gameweeks.at[i,'team_h'] \
                           for i in Gameweeks.index]
Gameweeks['side'] = ['home' if Gameweeks.at[i,'team'] == Gameweeks.at[i,'team_h']\
                            else 'away' \
                           for i in Gameweeks.index]

del Gameweeks['team_a']
del Gameweeks['team_h']

Gameweeks

,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,in_dreamteam,influence,...,red_cards,saves,threat,total_points,yellow_cards,id,gameweek,team,teamAgainst,side
1014,0,1,27,1,0.0,0,0,1.8,False,18.0,...,0,2,0.0,7,0,14,1,1,13,away
1027,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0,0.0,0,0,27,1,2,17,away
1047,0,2,29,1,0.0,0,0,2.5,False,25.4,...,0,3,0.0,9,0,47,1,4,18,away
1048,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0,0.0,0,0,48,1,4,18,away
1072,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0,0.0,0,0,72,1,3,15,home
1073,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0,0.0,0,0,73,1,3,15,home
1074,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0,0.0,0,0,74,1,3,15,home
1093,0,1,28,1,10.0,0,0,3.6,False,25.6,...,0,3,0.0,8,0,93,1,5,16,home
1094,0,0,15,0,0.0,3,0,2.4,False,24.2,...,0,3,0.0,2,0,94,1,2,17,away
1095,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0,0.0,0,0,95,1,5,16,home


In [15]:
Gameweeks

,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,in_dreamteam,influence,...,saves,threat,total_points,yellow_cards,id,gameweek,team,team_a,team_h,teamAgainst
1014,0,1,27,1,0.0,0,0,1.8,False,18.0,...,2,0.0,7,0,14,1,1,1,13,13
1027,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0.0,0,0,27,1,2,2,17,17
1047,0,2,29,1,0.0,0,0,2.5,False,25.4,...,3,0.0,9,0,47,1,4,4,18,18
1048,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0.0,0,0,48,1,4,4,18,18
1072,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0.0,0,0,72,1,3,15,3,15
1073,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0.0,0,0,73,1,3,15,3,15
1074,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0.0,0,0,74,1,3,15,3,15
1093,0,1,28,1,10.0,0,0,3.6,False,25.6,...,3,0.0,8,0,93,1,5,16,5,16
1094,0,0,15,0,0.0,3,0,2.4,False,24.2,...,3,0.0,2,0,94,1,2,2,17,17
1095,0,0,0,0,0.0,0,0,0.0,False,0.0,...,0,0.0,0,0,95,1,5,16,5,16


In [17]:
for i in range(len(Gameweeks)):
    A = Fixtures[Fixtures['event'] == Gameweeks.iloc[13,20]]
    Gameweeks.loc[i,'team_a'] = A[(A['team_a'] == Gameweeks.iloc[13,21])|(A['team_h'] == Gameweeks.iloc[13,21])]['team_a']
Gameweeks

ValueError: Incompatible indexer with Series

In [25]:
Gameweeks.at[1001,'id']

1.0

In [1]:
from bs4 import BeautifulSoup
import requests
import ast
import pandas as pd
import json
url1 = "https://fantasy.premierleague.com/api/bootstrap-static/"
url2 = "https://fantasy.premierleague.com/api/entry/698498/history/"
url3 = "https://fantasy.premierleague.com/api/event/6/live/"
url4 = "https://fantasy.premierleague.com/api/fixtures"
p = requests.get(url1)
page = BeautifulSoup(p.text)
data = str(page.p)[3:-4]

#import ast
#d = ast.literal_eval(data)

d = json.loads(data)

table = pd.DataFrame(d['elements'])

#pdcurrent.index = pdcurrent['event']
#del pdcurrent['event']
table = table[['team', 'element_type', 'web_name', 'goals_scored', 'assists', 'bonus', 'event_points', 'total_points', 
               'saves', 'own_goals', 'clean_sheets', 'penalties_missed', 'penalties_saved', 'yellow_cards', 'red_cards', 
               'minutes', 'bps', 'creativity', 'threat', 'ict_index', 'influence',
               'value_season', 'form', 'value_form', 'points_per_game', 
               'goals_conceded', 
               'in_dreamteam', 'dreamteam_count',
               'now_cost', 'cost_change_event', 'cost_change_event_fall',
               'cost_change_start', 'cost_change_start_fall', 'selected_by_percent',
               'transfers_in_event', 'transfers_out_event', 'transfers_in', 'transfers_out', 
               'chance_of_playing_this_round', 'chance_of_playing_next_round', 'news_added', 'news', 'status', 
               'ep_this', 'ep_next', 'first_name', 'second_name', 'team_code', 'id', 'photo', 'special', 'squad_number', 'code']]
table.to_excel('fpltable.xlsx')

In [13]:
from bs4 import BeautifulSoup
import requests
import ast
import pandas as pd
import json
url1 = "https://fantasy.premierleague.com/api/bootstrap-static/"
url2 = "https://fantasy.premierleague.com/api/entry/698498/history/"
url3 = "https://fantasy.premierleague.com/api/event/1/live/"
url4 = "https://fantasy.premierleague.com/api/fixtures"
p = requests.get(url3)
page = BeautifulSoup(p.text)
data = str(page.p)[3:-4]

#import ast
#d = ast.literal_eval(data)

d = json.loads(data)
d.keys()
#table = pd.DataFrame(d)
#table.to_excel('fplfixtures.xlsx')
#pd.DataFrame(d)
pd.DataFrame(d['elements'])

,explain,id,stats
0,"[{'fixture': 10, 'stats': [{'identifier': 'min...",14,"{'minutes': 90, 'goals_scored': 0, 'assists': ..."
1,"[{'fixture': 6, 'stats': [{'identifier': 'minu...",27,"{'minutes': 0, 'goals_scored': 0, 'assists': 0..."
2,"[{'fixture': 7, 'stats': [{'identifier': 'minu...",47,"{'minutes': 90, 'goals_scored': 0, 'assists': ..."
3,"[{'fixture': 7, 'stats': [{'identifier': 'minu...",48,"{'minutes': 0, 'goals_scored': 0, 'assists': 0..."
4,"[{'fixture': 2, 'stats': [{'identifier': 'minu...",72,"{'minutes': 0, 'goals_scored': 0, 'assists': 0..."
5,"[{'fixture': 2, 'stats': [{'identifier': 'minu...",73,"{'minutes': 0, 'goals_scored': 0, 'assists': 0..."
6,"[{'fixture': 2, 'stats': [{'identifier': 'minu...",74,"{'minutes': 0, 'goals_scored': 0, 'assists': 0..."
7,"[{'fixture': 3, 'stats': [{'identifier': 'minu...",93,"{'minutes': 90, 'goals_scored': 0, 'assists': ..."
8,"[{'fixture': 6, 'stats': [{'identifier': 'minu...",94,"{'minutes': 90, 'goals_scored': 0, 'assists': ..."
9,"[{'fixture': 3, 'stats': [{'identifier': 'minu...",95,"{'minutes': 0, 'goals_scored': 0, 'assists': 0..."


In [61]:
d1.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [74]:
pd.DataFrame(d1['teams'])['id']
pd.DataFrame(d1['teams'])['name']
teams = dict(zip(pd.DataFrame(d1['teams'])['id'],pd.DataFrame(d1['teams'])['name']))
players = dict(zip(bigTable['id'],bigTable['first_name']+' '+bigTable['second_name']))

{1: 'Shkodran Mustafi',
 2: 'Héctor Bellerín',
 3: 'Sead Kolasinac',
 4: 'Ainsley Maitland-Niles',
 5: 'Sokratis Papastathopoulos',
 6: 'Nacho Monreal',
 7: 'Laurent Koscielny',
 8: 'Konstantinos Mavropanos',
 9: 'Carl Jenkinson',
 10: 'Rob Holding',
 11: 'Pierre-Emerick Aubameyang',
 12: 'Alexandre Lacazette',
 13: 'Edward Nketiah',
 14: 'Bernd Leno',
 15: 'Mesut Özil',
 16: 'Henrikh Mkhitaryan',
 18: 'Granit Xhaka',
 19: 'Lucas Torreira',
 20: 'Mohamed Elneny',
 21: 'Matteo Guendouzi',
 106: 'David Luiz Moreira Marinho',
 427: 'Emiliano Martínez',
 467: 'Calum Chambers',
 469: 'Daniel Ceballos Fernández',
 488: 'Nicolas Pépé',
 489: 'Reiss Nelson',
 490: 'Joseph Willock',
 504: 'Gabriel Teodoro Martinelli Silva',
 515: 'Kieran Tierney',
 541: 'Bukayo Saka',
 22: 'Ahmed El Mohamady',
 23: 'James Chester',
 24: 'Neil Taylor',
 25: 'Kortney Hause',
 26: 'Jonathan Kodjia',
 27: 'Ørjan Nyland',
 28: 'Conor Hourihane',
 29: 'Jack Grealish',
 30: 'Anwar El Ghazi',
 31: 'John McGinn',
 32: '

In [92]:
[{'identifier': 'goals_scored', 'a': [{'value': 1, 'element': 278}], 'h': [{'value': 1, 'element': 183}, {'value': 1, 'element': 188}, {'value': 1, 'element': 191}]}, {'identifier': 'assists', 'a': [{'value': 1, 'element': 283}], 'h': [{'value': 1, 'element': 182}, {'value': 1, 'element': 187}, {'value': 1, 'element': 188}, {'value': 1, 'element': 191}]}, {'identifier': 'own_goals', 'a': [{'value': 1, 'element': 277}], 'h': []}, {'identifier': 'penalties_saved', 'a': [], 'h': []}, {'identifier': 'penalties_missed', 'a': [], 'h': []}, {'identifier': 'yellow_cards', 'a': [{'value': 1, 'element': 283}, {'value': 1, 'element': 288}], 'h': []}, {'identifier': 'red_cards', 'a': [], 'h': []}, {'identifier': 'saves', 'a': [{'value': 4, 'element': 280}], 'h': [{'value': 2, 'element': 189}, {'value': 2, 'element': 526}]}, {'identifier': 'bonus', 'a': [{'value': 1, 'element': 278}], 'h': [{'value': 3, 'element': 188}, {'value': 2, 'element': 191}]}, {'identifier': 'bps', 'a': [{'value': 32, 'element': 278}, {'value': 19, 'element': 280}, {'value': 18, 'element': 273}, {'value': 16, 'element': 274}, {'value': 16, 'element': 283}, {'value': 15, 'element': 276}, {'value': 12, 'element': 286}, {'value': 11, 'element': 290}, {'value': 10, 'element': 287}, {'value': 3, 'element': 434}, {'value': 2, 'element': 277}, {'value': 1, 'element': 285}, {'value': -3, 'element': 288}], 'h': [{'value': 46, 'element': 188}, {'value': 40, 'element': 191}, {'value': 27, 'element': 183}, {'value': 26, 'element': 187}, {'value': 24, 'element': 182}, {'value': 18, 'element': 197}, {'value': 15, 'element': 181}, {'value': 12, 'element': 198}, {'value': 11, 'element': 199}, {'value': 10, 'element': 184}, {'value': 7, 'element': 189}, {'value': 7, 'element': 526}, {'value': 3, 'element': 192}, {'value': 3, 'element': 200}]}]

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [31]:
table.loc[1,'stats']

[{'identifier': 'goals_scored',
  'a': [{'value': 1, 'element': 278}],
  'h': [{'value': 1, 'element': 183},
   {'value': 1, 'element': 188},
   {'value': 1, 'element': 191}]},
 {'identifier': 'assists',
  'a': [{'value': 1, 'element': 283}],
  'h': [{'value': 1, 'element': 182},
   {'value': 1, 'element': 187},
   {'value': 1, 'element': 188},
   {'value': 1, 'element': 191}]},
 {'identifier': 'own_goals', 'a': [{'value': 1, 'element': 277}], 'h': []},
 {'identifier': 'penalties_saved', 'a': [], 'h': []},
 {'identifier': 'penalties_missed', 'a': [], 'h': []},
 {'identifier': 'yellow_cards',
  'a': [{'value': 1, 'element': 283}, {'value': 1, 'element': 288}],
  'h': []},
 {'identifier': 'red_cards', 'a': [], 'h': []},
 {'identifier': 'saves',
  'a': [{'value': 4, 'element': 280}],
  'h': [{'value': 2, 'element': 189}, {'value': 2, 'element': 526}]},
 {'identifier': 'bonus',
  'a': [{'value': 1, 'element': 278}],
  'h': [{'value': 3, 'element': 188}, {'value': 2, 'element': 191}]},
 {'i

In [117]:
pd.DataFrame(d['elements']).columns

Index(['assists', 'bonus', 'bps', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'clean_sheets', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'creativity', 'dreamteam_count',
       'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name',
       'form', 'goals_conceded', 'goals_scored', 'ict_index', 'id',
       'in_dreamteam', 'influence', 'minutes', 'news', 'news_added',
       'now_cost', 'own_goals', 'penalties_missed', 'penalties_saved', 'photo',
       'points_per_game', 'red_cards', 'saves', 'second_name',
       'selected_by_percent', 'special', 'squad_number', 'status', 'team',
       'team_code', 'threat', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'yellow_cards'],
      dtype='object')

In [129]:
table = pd.DataFrame(d['elements'])

#pdcurrent.index = pdcurrent['event']
#del pdcurrent['event']
table = table[['team', 'element_type', 'web_name', 'goals_scored', 'assists', 'bonus', 'event_points', 'total_points', 
               'saves', 'own_goals', 'clean_sheets', 'penalties_missed', 'penalties_saved', 'yellow_cards', 'red_cards', 
               'minutes', 'bps', 'creativity', 'threat', 'ict_index', 'influence',
               'value_season', 'form', 'value_form', 'points_per_game', 
               'goals_conceded', 
               'in_dreamteam', 'dreamteam_count',
               'now_cost', 'cost_change_event', 'cost_change_event_fall',
               'cost_change_start', 'cost_change_start_fall', 'selected_by_percent',
               'transfers_in_event', 'transfers_out_event', 'transfers_in', 'transfers_out', 
               'chance_of_playing_this_round', 'chance_of_playing_next_round', 'news_added', 'news', 'status', 
               'ep_this', 'ep_next', 'first_name', 'second_name', 'team_code', 'id', 'photo', 'special', 'squad_number', 'code']]
table.to_excel('fpltable.xlsx')
table['creativity'] = table['creativity'].astype(float)
table.iloc[142]['creativity']
table.sort_values(by='creativity', ascending=False)

,team,element_type,web_name,goals_scored,assists,bonus,event_points,total_points,saves,own_goals,...,ep_this,ep_next,first_name,second_name,team_code,id,photo,special,squad_number,code
278,11,3,De Bruyne,2,7,9,17,54,0,0,...,10.5,10.0,Kevin,De Bruyne,43,215,61366.jpg,False,None,61366
242,10,2,Alexander-Arnold,0,3,2,0,24,0,0,...,5.0,6.0,Trent,Alexander-Arnold,14,182,169187.jpg,False,None,169187
505,19,3,Lanzini,0,2,3,0,21,0,0,...,1.7,2.1,Manuel,Lanzini,21,391,86934.jpg,False,None,86934
98,4,3,Groß,0,1,1,3,16,0,0,...,1.3,0.8,Pascal,Groß,36,49,60307.jpg,False,None,60307
200,8,3,Sigurdsson,0,1,0,2,17,0,0,...,3.3,1.8,Gylfi,Sigurdsson,11,151,55422.jpg,False,None,55422
361,14,3,Buendía,0,4,3,2,26,0,0,...,5.0,5.0,Emiliano,Buendía,45,283,195546.jpg,False,None,195546
280,11,3,Mahrez,1,5,3,15,35,0,0,...,6.5,6.0,Riyad,Mahrez,43,217,103025.jpg,False,None,103025
190,8,2,Digne,0,2,3,1,24,0,0,...,2.3,0.8,Lucas,Digne,11,141,101188.jpg,False,None,101188
226,9,3,Maddison,1,2,4,10,27,0,0,...,4.3,3.6,James,Maddison,13,171,172780.jpg,False,None,172780
388,15,3,Norwood,0,1,0,5,16,0,0,...,2.3,1.3,Oliver,Norwood,49,302,79934.jpg,False,None,79934


In [10]:
#dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])
pd.DataFrame(d['element_types'])

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_max_play,squad_min_play,squad_select,sub_positions_locked,ui_shirt_specific
0,1,Goalkeepers,GKP,Goalkeeper,GKP,1,1,2,[12],True
1,2,Defenders,DEF,Defender,DEF,5,3,5,[],False
2,3,Midfielders,MID,Midfielder,MID,5,2,5,[],False
3,4,Forwards,FWD,Forward,FWD,3,1,3,[],False


In [3]:
pd.DataFrame(d['events'])

,average_entry_score,chip_plays,data_checked,deadline_time,deadline_time_epoch,deadline_time_game_offset,finished,highest_score,highest_scoring_entry,id,...,is_next,is_previous,most_captained,most_selected,most_transferred_in,most_vice_captained,name,top_element,top_element_info,transfers_made
0,65,"[{'chip_name': 'bboost', 'num_played': 128770}...",False,2019-08-09T18:00:00Z,1565373600,0,True,142.0,3493085.0,1,...,False,False,191.0,183.0,1.0,189.0,Gameweek 1,214.0,"{'id': 214, 'points': 20}",0
1,41,"[{'chip_name': 'bboost', 'num_played': 114585}...",False,2019-08-17T10:30:00Z,1566037800,0,True,119.0,6106693.0,2,...,False,False,191.0,183.0,235.0,214.0,Gameweek 2,278.0,"{'id': 278, 'points': 17}",9998839
2,44,"[{'chip_name': 'bboost', 'num_played': 79958},...",False,2019-08-23T18:00:00Z,1566583200,0,True,107.0,5087809.0,3,...,False,False,214.0,183.0,278.0,191.0,Gameweek 3,191.0,"{'id': 191, 'points': 15}",13387638
3,57,"[{'chip_name': 'bboost', 'num_played': 72741},...",False,2019-08-31T10:30:00Z,1567247400,0,True,136.0,6372519.0,4,...,False,False,214.0,183.0,278.0,191.0,Gameweek 4,166.0,"{'id': 166, 'points': 16}",13215999
4,52,"[{'chip_name': 'bboost', 'num_played': 58835},...",False,2019-09-14T10:30:00Z,1568457000,0,True,128.0,6223449.0,5,...,False,False,191.0,183.0,243.0,214.0,Gameweek 5,342.0,"{'id': 342, 'points': 16}",12363678
5,52,"[{'chip_name': 'bboost', 'num_played': 47670},...",False,2019-09-20T18:00:00Z,1569002400,0,True,141.0,3423365.0,6,...,False,False,214.0,183.0,460.0,191.0,Gameweek 6,218.0,"{'id': 218, 'points': 19}",7987314
6,51,"[{'chip_name': 'bboost', 'num_played': 65678},...",False,2019-09-28T10:30:00Z,1569666600,0,True,118.0,6758371.0,7,...,False,False,191.0,183.0,460.0,215.0,Gameweek 7,401.0,"{'id': 401, 'points': 15}",9336557
7,36,"[{'chip_name': 'bboost', 'num_played': 54626},...",False,2019-10-05T10:30:00Z,1570271400,0,True,100.0,6815331.0,8,...,False,True,214.0,183.0,217.0,191.0,Gameweek 8,417.0,"{'id': 417, 'points': 16}",7457686
8,23,"[{'chip_name': 'bboost', 'num_played': 57241},...",False,2019-10-19T10:30:00Z,1571481000,0,False,92.0,6919656.0,9,...,False,False,210.0,183.0,192.0,214.0,Gameweek 9,103.0,"{'id': 103, 'points': 14}",8197899
9,0,[],False,2019-10-25T18:00:00Z,1572026400,0,False,NaN,NaN,10,...,True,False,NaN,NaN,NaN,NaN,Gameweek 10,NaN,None,0


In [49]:
import requests
session = requests.session()

In [50]:
url = 'https://users.premierleague.com/accounts/login/'
payload = {
 'password': 'mmmm57',
 'login': 'constti@gmail.com',
 'redirect_uri': 'https://fantasy.premierleague.com/a/login',
 'app': 'plfpl-web'
}
session.post(url, data=payload)

<Response [200]>

In [52]:
response = session.get('https://fantasy.premierleague.com/drf/my-team/698498')
response

<Response [200]>

In [11]:
import pandas as pd
d.keys()
df = pd.DataFrame(list(d.keys()))
df

,0
0,events
1,game_settings
2,phases
3,teams
4,total_players
5,elements
6,element_stats
7,element_types


In [75]:
pdcurrent = pd.DataFrame(d['current'])
pdpast = pd.DataFrame(d['past'])
pdchips = pd.DataFrame(d['chips'])

pdcurrent.index = pdcurrent['event']
del pdcurrent['event']
pdcurrent = pdcurrent[['event_transfers', 'points', "points_on_bench", 'rank', 'total_points', 'value', 'bank']]
pdcurrent.to_excel('fpltable.xlsx')
pdcurrent

,event_transfers,points,points_on_bench,rank,total_points,value,bank
event,,,,,,,
1,0,96,14,109262,96,1000,25
2,0,57,7,459677,153,1002,25
3,3,57,5,1195725,206,1007,3
4,2,52,4,4185847,254,1010,5
5,0,48,6,4125678,302,1013,5
6,2,43,14,1447623,302,1014,13
